In [1]:
import numpy as np
import numpy.typing as npt
import gym

from typing import List, Tuple, Literal, Any, Optional, cast, Callable
import plotly.graph_objects as go
from utils.agent import Agent
from tqdm.autonotebook import tqdm
from utils.algorithm import AlgorithmInterface
from utils.preprocess import PreprocessInterface
import torch
from collections import deque
from torchvision import transforms
import math
from torch import nn
import sys
from copy import deepcopy
from utils.common import Step, Episode, TransitionGeneric


In [2]:
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
env = gym.make("Pong-v0")
env.seed(RANDOM_SEED)
env.reset()
env._max_episode_steps = 1_0000
TOTAL_ACTIONS = env.action_space.n


A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [5]:
TOTAL_ACTIONS

6

In [6]:
# shape is (210, 160, 3)
Observation = npt.NDArray[np.uint8]
Action = int

# shape is (4, 210, 160, 3)
State = torch.Tensor
Reward = int

Transition = TransitionGeneric[State, Action]

In [7]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(4, 32, (8, 8), 4),
            nn.ReLU(),
            nn.Conv2d(32, 64, (4, 4), 2),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3, 3), 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(7 * 7 * 64, 512),
            nn.Linear(512, TOTAL_ACTIONS),
        ).to(device)

    def forward(self, x: State) -> torch.Tensor:
        rlt = cast(torch.Tensor, self.net(x.to(device)))
        assert rlt.shape == (x.shape[0], TOTAL_ACTIONS)
        return rlt.cpu()


In [8]:
class RandomAlgorithm(AlgorithmInterface[State, Action]):
    def __init__(self):
        # self.after_step_freq = 1
        # self.need_on_termination = True
        self.frame_skip = 0
        self.reset()

    def reset(self):
        self.times = 1
        self.last_action = None

    def allowed_actions(self, state: State) -> List[Action]:
        return list(range(TOTAL_ACTIONS))

    def take_action(self, state: State) -> Action:
        self.times += 1

        if self.times % 10 == 0:
            act = np.random.choice(self.allowed_actions(state))
            self.last_action = act
            return act

        if self.last_action is not None:
            return self.last_action

        act = np.random.choice(self.allowed_actions(state))
        self.last_action = act
        return act

    def after_step(
        self,
        sar: Tuple[State, Action, Reward],
        sa: Tuple[State, Optional[Action]],
    ):
        pass


In [9]:
DEFAULT_TRAINING_TIMES = 12_50_0000


class NNAlgorithm(AlgorithmInterface[State, Action]):
    def __init__(self, nn: DQN, training_times: int = 12_50_0000, gamma: float = 0.99):
        self.frame_skip = 3

        self.times = 0

        self.network = nn
        self.optimizer = torch.optim.RMSprop(
            self.network.parameters(), 1e-3, 0.95, 0.01
        )

        self.shrink = min(training_times / DEFAULT_TRAINING_TIMES, 1)
        if self.shrink != 1:
            print(f"training on shrinked mode: {self.shrink}")

        self.target_network = DQN()
        self.target_network.load_state_dict(self.network.state_dict())

        self.batch_size = 32

        self.update_target = 1000

        self.memory_replay: deque[Transition] = deque(
            maxlen=math.ceil(5_0000 * self.shrink)
        )
        self.gamma = gamma
        self.loss_func = torch.nn.MSELoss()

    def reset(self):
        pass

    def allowed_actions(self, _: State) -> List[Action]:
        return list(range(TOTAL_ACTIONS))

    def take_action(self, state: State) -> Action:
        rand = np.random.random()
        max_decry_times = 100_0000 * self.shrink
        sigma = 1 - 0.9 / max_decry_times * np.min([self.times, max_decry_times])
        if rand < sigma:
            return np.random.choice(self.allowed_actions(state))

        else:
            act_vals: torch.Tensor = self.network(state)
            maxi = torch.argmax(act_vals)
            return cast(Action, maxi)

    def after_step(
        self,
        sar: Tuple[State, Action, Reward],
        sa: Tuple[State, Optional[Action]],
    ):
        (s, a, r) = sar
        (sn, an) = sa
        self.memory_replay.append((s, a, r, sn, an))

        if len(self.memory_replay) >= 1.25 * self.batch_size:

            batch: List[Transition] = cast(List[Transition], [None] * self.batch_size)
            for idx, i in enumerate(
                np.random.choice(len(self.memory_replay), self.batch_size)
            ):
                batch[idx] = self.memory_replay[i]

            self.train(batch)

        if self.times != 0 and self.times % (self.update_target) == 0:
            self.update_target_network()

        self.times += 1

    def update_target_network(self):
        self.target_network.load_state_dict(self.network.state_dict())

    def clip_reward(self, r: float) -> float:
        if r > 0:
            return 1.0
        elif r < 0:
            return -1.0
        else:
            return 0

    def train(self, batch: List[Transition]):

        masks = torch.tensor(
            [0 if an is None else 1 for (_, _, _, _, an) in batch],
            dtype=torch.float,
        )

        target = torch.tensor(
            [self.clip_reward(r) for (_, _, r, _, _) in batch], dtype=torch.float
        ) + torch.inner(
            masks,
            self.gamma
            * torch.max(
                self.network(torch.cat([sn for (_, _, _, sn, _) in batch])), dim=1
            )[0],
        )

        assert target.shape == (32,)
        x_vals = self.network(torch.cat([s for (s, _, _, _, _) in batch]))
        x = x_vals[range(x_vals.shape[0]), [a for (_, a, _, _, _) in batch]]

        assert x.shape == (32,)

        loss = self.loss_func(x, target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def on_termination(self, sar: Tuple[List[State], List[Action], List[Reward]]):
        pass


class Preprocess(PreprocessInterface[Observation, Action, State]):
    def __init__(self):
        self.trfm: Callable[[Observation], State] = transforms.Compose(
            [transforms.ToTensor(), transforms.Grayscale(), transforms.Resize((84, 84))]
        )
        self.reset()

    def reset(self):
        pass

    def get_current_state(self, h: List[Observation]) -> State:
        assert len(h) > 0

        last_4_arr = self.stack_4(h, -1)

        rlt = torch.cat([self.trfm(i) for i in last_4_arr]).unsqueeze(0)
        assert rlt.shape == (1, 4, 84, 84)
        return rlt

    def stack_4(self, h: List[Observation], idx: int) -> npt.NDArray[np.uint8]:

        assert idx < 0
        last_4_index = [-3 + idx, -2 + idx, -1 + idx, idx]

        last_4: List[Observation] = []
        for idx in last_4_index:
            if -idx <= len(h):
                last_4.append(np.asarray((h[idx])))

        last_4_arr = np.asarray(last_4)
        while last_4_arr.shape[0] < 4:
            last_4_arr = np.insert(last_4_arr, 0, last_4[0], axis=0)

        assert last_4_arr.shape == (4, 210, 160, 3)

        return last_4_arr


In [10]:
TRAINING_TIMES = DEFAULT_TRAINING_TIMES
# TRAINING_TIMES = 2_0000

agent = Agent(env, NNAlgorithm(DQN(), TRAINING_TIMES), Preprocess())
training_rwds: List[float] = []

with tqdm(range(4200)) as pbar:
    for _ in pbar:

        # pbar.update(1)
        # frames = 1
        agent.reset(["preprocess"])
        end = False
        # while frames < TRAINING_TIMES:
        while not end:

            # end = False

            # while not end and frames < TRAINING_TIMES:
            (_, end) = agent.step()

            # agent.render('human')
            # frames += 1
            # pbar.update(1)

        training_rwds.append(
            np.sum([r for r in agent.episode_reward])
        )
        pbar.set_postfix(rwd=training_rwds[-1])


 36%|███▌      | 1508/4200 [1:13:57<2:12:02,  2.94s/it, rwd=-20]


KeyboardInterrupt: 

In [ ]:
np.save("./training.arr", np.asarray(training_rwds))


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=[i + 1 for i in range(len(training_rwds))],
               y = [r for r in training_rwds])
)
# fig.update_yaxes(type="log")
# fig.update_layout(yaxis_type="log")
fig.show()



In [ ]:
EVALUATION_TIMES = 30
MAX_EPISODE_LENGTH = 18_000
rwds: List[int] = []
agent.toggleEval(False)

for _ in tqdm(range(EVALUATION_TIMES)):
    agent.reset(['preprocess'])

    end = False
    i = 1

    while not end and i < MAX_EPISODE_LENGTH:
        (o, end) = agent.step()
        i += 1
        env.render()
        # if end:
        #     rwds.append(np.sum([r if r is not None else 0 for (_,
        #                                                        _, r) in cast(Episode, episode)]))
    rwds.append(
        np.sum([r for r in agent.episode_reward])
    )


  0%|          | 0/30 [00:00<?, ?it/s]/home/diyuan/.local/lib/python3.9/site-packages/gym/envs/atari/environment.py:267: UserWarning:

WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.

100%|██████████| 30/30 [00:33<00:00,  1.10s/it]


In [ ]:
np.save("./eval.arr", np.asarray(rwds))

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=[i + 1 for i in range(len(rwds))],
               y = [r for r in rwds])
)
# fig.update_yaxes(type="log")
# fig.update_layout(yaxis_type="log")
fig.show()

